In [1]:
import warnings
warnings.filterwarnings('ignore')

import math
import numpy as np
import pandas as pd
from collections import Counter

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

import seaborn as sns; sns.set(style="ticks", color_codes=True)

import scipy.stats as scs
import statistics as stats

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from scipy.sparse import csr_matrix

In [3]:
# import nltk
# import string
# from nltk.corpus import stopwords
# from nltk.tokenize import sent_tokenize, word_tokenize

In [4]:
# nltk.download('punkt')
# nltk.download('stopwords')

In [5]:
# pip install pyspark

In [6]:
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import Row

# Setup a SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [7]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [9]:
base_df = pd.read_csv('data/rtr_data.csv')
base_df.drop(columns='Unnamed: 0', inplace=True)

In [10]:
base_df

,fit,user_id,bust_size,item_id,weight_lbs,rating,rented_for,review_text,body_type,review_summary,category,height_inches,size,age,review_date,review-summary,keywords
0,fit,420272,34d,2260466,137,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,68.0,14,28.0,2016-04-20,An adorable romper! Belt and zipper were a lit...,"['adorable', 'romper', 'belt', 'zipper', 'litt..."
1,fit,273551,34b,153475,132,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,66.0,12,36.0,2013-06-18,I rented this dress for a photo shoot. The the...,"['photo', 'shoot', 'theme', 'hollywood', 'glam..."
2,fit,909926,34c,126335,135,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,65.0,8,34.0,2014-02-12,I rented this for my company's black tie award...,"['company', 'black', 'tie', 'awards', 'banquet..."
3,fit,151944,34b,616682,145,10.0,wedding,I have always been petite in my upper body and...,athletic,Was in love with this dress !!!,gown,69.0,12,27.0,2016-09-26,I have always been petite in my upper body and...,"['always', 'petite', 'upper', 'body', 'extreme..."
4,fit,734848,32b,364092,138,8.0,date,Didn't actually wear it. It fit perfectly. The...,athletic,Traditional with a touch a sass,dress,68.0,8,45.0,2016-04-30,Didn't actually wear it. It fit perfectly. The...,"['actually', 'perfectly', 'fabric', 'little', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146132,fit,66386,34dd,2252812,140,10.0,work,Fit like a glove!,hourglass,LOVE IT!!! First Item Im thinking of buying!,jumpsuit,69.0,8,42.0,2016-05-18,Fit like a glove! LOVE IT!!! First Item Im thi...,"['like', 'glove', 'love', 'first', 'item', 'im..."
146133,fit,118398,32c,682043,100,10.0,work,The pattern contrast on this dress is really s...,petite,LOVE it!,dress,61.0,4,29.0,2016-09-30,The pattern contrast on this dress is really s...,"['pattern', 'contrast', 'really', 'stunning', ..."
146134,fit,47002,36a,683251,135,6.0,everyday,"Like the other DVF wraps, the fit on this is f...",straight & narrow,"Loud patterning, flattering fit",dress,68.0,8,31.0,2016-03-04,"Like the other DVF wraps, the fit on this is f...","['like', 'dvf', 'wraps', 'fantastic', 'albeit'..."
146135,fit,961120,36c,126335,165,10.0,wedding,This dress was PERFECTION. it looked incredib...,pear,loved this dress it was comfortable and photog...,dress,66.0,16,31.0,2015-11-25,This dress was PERFECTION. it looked incredib...,"['perfection', 'looked', 'incredible', 'photos..."


In [11]:
ratings_df = base_df[['user_id', 'item_id', 'rating']]

In [12]:
spark_ratings = spark.createDataFrame(ratings_df)

In [16]:
train, test = spark_ratings.randomSplit([0.7, 0.3])

In [48]:
als_model = ALS(
    itemCol='item_id',
    userCol='user_id',
    ratingCol='rating',
    nonnegative=True,    
    regParam=0.75,
    rank=5,  # 5
    coldStartStrategy='nan')

# als_model = ALS(
#     itemCol='item_id',
#     userCol='user_id',
#     ratingCol='rating',
#     nonnegative=True,    
#     regParam=0.1,
#     rank=10) 

TypeError: Invalid param value given for param "coldStartStrategy". Could not convert <class 'int'> to string type

In [46]:
recommender = als_model.fit(train)

In [37]:
# test_pred = recommender.transform(test)

In [34]:
test_pred.show()

+-------+-------+------+----------+
|user_id|item_id|rating|prediction|
+-------+-------+------+----------+
| 476519| 213089|   4.0|  9.863154|
| 371542| 213089|  10.0|    8.7733|
| 768039| 213089|  10.0|   10.1906|
| 634637| 213089|  10.0|  8.876076|
| 106247| 213089|  10.0| 7.4793615|
| 904529| 277366|  10.0|       NaN|
| 656376| 277366|  10.0| 7.6747227|
| 978145| 277366|  10.0| 5.9935484|
| 524777| 277366|  10.0|       NaN|
| 503448| 277366|  10.0|  9.380051|
| 742258| 277366|  10.0|       NaN|
| 712415| 333479|   8.0|  9.135224|
| 260350| 333479|  10.0| 10.122275|
| 585785| 333479|  10.0| 10.282008|
| 469902| 640839|  10.0|  9.342384|
| 796310| 640839|  10.0| 3.7439528|
| 941223| 640839|   8.0|  8.318035|
| 430980| 640839|   8.0|       NaN|
| 642748| 640839|  10.0|  8.907951|
| 210577| 640839|  10.0|  7.904005|
+-------+-------+------+----------+
only showing top 20 rows



In [47]:
new_pred = recommender.transform(test)
new_pred.show()

+-------+-------+------+----------+
|user_id|item_id|rating|prediction|
+-------+-------+------+----------+
| 476519| 213089|   4.0|  8.070737|
| 371542| 213089|  10.0|  8.829331|
| 768039| 213089|  10.0| 7.0345416|
| 634637| 213089|  10.0|  9.428821|
| 106247| 213089|  10.0| 7.9476357|
| 904529| 277366|  10.0|       NaN|
| 656376| 277366|  10.0|  8.528872|
| 978145| 277366|  10.0| 7.4102626|
| 524777| 277366|  10.0|       NaN|
| 503448| 277366|  10.0|  8.839126|
| 742258| 277366|  10.0|       NaN|
| 712415| 333479|   8.0|  8.755282|
| 260350| 333479|  10.0| 7.2034793|
| 585785| 333479|  10.0|  9.006945|
| 469902| 640839|  10.0|  8.103055|
| 796310| 640839|  10.0| 5.6607285|
| 941223| 640839|   8.0|  8.725805|
| 430980| 640839|   8.0|       NaN|
| 642748| 640839|  10.0|  9.208943|
| 210577| 640839|  10.0|  9.096522|
+-------+-------+------+----------+
only showing top 20 rows

